# Ray RLlib - Sample Application: FrozenLake-v0

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

This example uses [RLlib](https://ray.readthedocs.io/en/latest/rllib.html) to train a policy with the `FrozenLake-v0` environment ([gym.openai.com/envs/FrozenLake-v0/](https://gym.openai.com/envs/FrozenLake-v0/)).

For more background about this problem, see:

* ["Introduction to Reinforcement Learning: the Frozen Lake Example"](https://reinforcementlearning4.fun/2019/06/09/introduction-reinforcement-learning-frozen-lake-example/), [Rodolfo Mendes](https://twitter.com/rodmsmendes)
* ["Gym Tutorial: The Frozen Lake"](https://reinforcementlearning4.fun/2019/06/16/gym-tutorial-frozen-lake/), [Rodolfo Mendes](https://twitter.com/rodmsmendes)

In [3]:
import ray
import ray.rllib.agents.ppo as ppo

In [4]:
ray.init(ignore_reinit_error=True)

2020-05-01 12:39:31,676	INFO resource_spec.py:212 -- Starting Ray with 4.3 GiB memory available for workers and up to 2.15 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-01 12:39:32,016	INFO services.py:1148 -- View the Ray dashboard at localhost:8268


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:22741',
 'object_store_address': '/tmp/ray/session_2020-05-01_12-39-31_663496_27220/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-05-01_12-39-31_663496_27220/sockets/raylet',
 'webui_url': 'localhost:8268',
 'session_dir': '/tmp/ray/session_2020-05-01_12-39-31_663496_27220'}

In [5]:
print(f'Dashboard URL: http://{ray.get_webui_url()}')

Dashboard URL: http://localhost:8268


Next we'll train an RLlib policy with the `FrozenLake-v0` environment.

By default, training runs for `10` iterations. Increase the `n_iter` setting if you want to see the resulting rewards improve.
Also note that *checkpoints* get saved after each iteration into the `/tmp/ppo/taxi` directory.

> **Note:** If you prefer to use a different directory root than `/tmp`, change it in the next cell **and** in the `rllib rollout` command below.

In [6]:
SELECT_ENV = "FrozenLake-v0"
N_ITER = 10

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

reward_history = []

agent = ppo.PPOTrainer(config, env=SELECT_ENV)

for _ in range(N_ITER):
    result = agent.train()
    print(result)

    max_reward = result["episode_reward_max"]
    reward_history.append(max_reward)

    file_name = agent.save("/tmp/ppo/froz")
    print(f'\ncheckpoint saved to {file_name}')

2020-05-01 12:40:18,645	INFO trainer.py:428 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-05-01 12:40:18,675	INFO trainer.py:585 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2020-05-01 12:40:20,935	INFO trainable.py:217 -- Getting current IP.
2020-05-01 12:40:20,936	WARNING util.py:37 -- Install gputil for GPU system monitoring.


(pid=30057) /Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=30057)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=30056) /Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=30056)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
{'episode_reward_max': 1.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 0.013232514177693762, 'episode_len_mean': 7.550094517958412, 'episodes_this_iter': 529, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [7]:
print(reward_history)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


Do the episode rewards increase after multiple iterations?
That shows whether the policy is improving.

Also, print out the policy and model to see the results of training in detail…

In [8]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())

print(model.base_model.summary())

[<tf.Variable 'default_policy/fc_1/kernel:0' shape=(16, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/kernel:0' shape=(16, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/kernel:0' shape=(256, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/kernel:0' shape=(256, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/kernel:0' shape=(256, 4) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/bias:0' shape=(4,) dtype=float32>,
 <tf.Variable 'default_policy/value_out/kernel:0' shape=(256, 1) dtype=float32>,
 <tf.Variable 'default_policy/value_out/bias:0' shape=(1,) dtype=float32>]
<tf.Tensor 'Reshape:0' shape=(?,) dtype=float32>
Model: "model"
________

Next we'll use the [`rollout` script](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies) to evaluate the trained policy.

This visualizes the "character" agent operating within the simulation: trying to find a walkable path to a goal tile.

In [9]:
! rllib rollout \
    /tmp/ppo/froz/checkpoint_2/checkpoint-2 \
    --config "{\"env\": \"FrozenLake-v0\"}" --run PPO \
    --steps 2000

2020-05-01 12:43:04,421	INFO resource_spec.py:212 -- Starting Ray with 4.1 GiB memory available for workers and up to 2.07 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-01 12:43:04,746	INFO services.py:1148 -- View the Ray dashboard at localhost:8269
2020-05-01 12:43:05,398	INFO trainer.py:428 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-05-01 12:43:05,428	INFO trainer.py:585 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2020-05-01 12:43:08,255	INFO trainable.py:217 -- Getting current IP.
2020-05-01 12:43:08,256	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020

The rollout uses the second saved checkpoint, evaluated through `2000` steps.
Modify the path to view other checkpoints.

Finally, launch [TensorBoard](https://ray.readthedocs.io/en/latest/rllib-training.html#getting-started) then follow the instructions (copy/paste the URL it generates) to visualize key metrics from training with RLlib…

In [ ]:
!tensorboard --logdir=~/ray_results/

Next, go through any of other remaining lessons:

* [04a: Application: Mountain Car](04a-Application-Mountain-Car.ipynb) -- Based on the `MountainCar-v0` environment from OpenAI Gym.
* [04b: Application: Taxi](04b-Application-Taxi.ipynb) -- Based on the `Taxi-v3` environment from OpenAI Gym.